## Loading data

In [1]:
import json
import requests
from rdflib import Graph
from requests.auth import HTTPDigestAuth

In [2]:
g = Graph()

In [3]:
g.parse("data/example-data.ttl", format="ttl")

<Graph identifier=Nd4e22c15a1e84633ad4b3af921bbc25d (<class 'rdflib.graph.Graph'>)>

## Use SPARQL Update

In [ ]:
# this works
from SPARQLWrapper import SPARQLWrapper
queryString = "INSERT DATA { GRAPH <https://golemlab.eu/data> { <https://golemlab.eu/data/1> a <https://golemlab.eu/data/corpus> . } }" 
sparql = SPARQLWrapper("http://localhost:8890/sparql-auth")
sparql.setHTTPAuth("DIGEST")
sparql.setCredentials("dba", "pwd123")
sparql.setQuery(queryString) 
sparql.method = 'POST' 
sparql.query()

documentation https://vos.openlinksw.com/owiki/wiki/VOS/VirtRDFInsert

## Upload a ttl file

this works:
```
curl --digest --user dba:pwd123 --url "http://localhost:8890/sparql-graph-crud-auth?graph=https://golemlab.eu/data" -X POST -T example-data.ttl
```

In [ ]:
ttl_content = g.serialize(format="ttl")

In [ ]:
graph_uri = "https://golemlab.eu/data"
url =  "http://localhost:8890/sparql-graph-crud-auth" + "?graph=" + graph_uri
print(url)
data = ttl_content.encode(encoding='utf-8')
response = requests.post(url=url, data=data, auth=HTTPDigestAuth("dba", "pwd123"), headers={'Content-Type': 'application/x-turtle'})
print(response.status_code)
print(response.text)

## Implemented in DB class

In [4]:
from sparql import DB
virtuoso = DB(triplestore="virtuoso", protocol="http",url="localhost",port="8890", username="dba", password="pwd123")

In [5]:
testdata = g.serialize()

In [6]:
virtuoso.crud_endpoint

'http://localhost:8890/sparql-graph-crud-auth'

In [7]:
virtuoso.upload(testdata, graph="https://golemlab.eu/data", format="ttl")

True

### delete the named graph

In [9]:
virtuoso.delete_graph("https://golemlab.eu/data")

True

```
PREFIX gd: <http://golemlab.eu/data/> 
PREFIX cls: <http://clscor.io/ontology/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?uri ?name ?acronym WHERE {
  ?uri a cls:X1_Corpus ;
crm:P1_is_identified_by ?nameID, ?acronymID .

?nameID crm:P2_has_type <http://golemlab.eu/data/entity/type/corpus_name> ; 
rdf:value ?name .

OPTIONAL {
?acronymID crm:P2_has_type <http://golemlab.eu/data/entity/type/corpus_acronym> ;
rdf:value ?acronym .
}

}
```